In [13]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from copy import deepcopy
from IPython.core.display import display, HTML

In [14]:
def top_ten_stations(df):
    
    df1 = df.groupby(['STATION', pd.Grouper(key='DATETIME', freq='1D')])\
    .sum()\
    .reset_index()\
    .groupby(['STATION'])\
    .mean()\
    .TOTAL_TRAFFIC\
    .sort_values(ascending=False)
    stations = df1.reset_index().STATION.unique()[:10]
    df2 = df[df.STATION.apply(lambda x: x in stations)][['STATION', 'DATETIME', 'TOTAL_TRAFFIC', 'DOW','TIME','BAD_WEATHER']]
    return df2

In [15]:
df = pd.read_csv('./final_2017.csv', index_col=0)
df['DATETIME'] = pd.to_datetime(df.DATETIME, 
                                    format="%Y/%m/%d %H:%M:%S")
df.info()

/home/rjh336/anaconda2/envs/metis/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1984948 entries, 0 to 1984947
Data columns (total 16 columns):
C/A              object
UNIT             object
SCP              object
STATION          object
DATETIME         datetime64[ns]
DOW              object
TIME             object
TOTAL_TRAFFIC    float64
DATE             object
NAME             object
PRCP             float64
SNOW             float64
SNWD             int64
TMAX             int64
TMIN             int64
BAD_WEATHER      int64
dtypes: datetime64[ns](1), float64(3), int64(4), object(8)
memory usage: 196.9+ MB


In [16]:
df.head()

,C/A,UNIT,SCP,STATION,DATETIME,DOW,TIME,TOTAL_TRAFFIC,DATE,NAME,PRCP,SNOW,SNWD,TMAX,TMIN,BAD_WEATHER
0,A002,R051,02-00-00,59 ST,2017-04-01 00:00:00,Saturday,04:00:00,36.0,2017-04-01,"NY CITY CENTRAL PARK, NY US",0.0,0.0,0,48,37,0
1,A002,R051,02-00-00,59 ST,2017-04-01 04:00:00,Saturday,08:00:00,51.0,2017-04-01,"NY CITY CENTRAL PARK, NY US",0.0,0.0,0,48,37,0
2,A002,R051,02-00-00,59 ST,2017-04-01 08:00:00,Saturday,12:00:00,227.0,2017-04-01,"NY CITY CENTRAL PARK, NY US",0.0,0.0,0,48,37,0
3,A002,R051,02-00-00,59 ST,2017-04-01 12:00:00,Saturday,16:00:00,324.0,2017-04-01,"NY CITY CENTRAL PARK, NY US",0.0,0.0,0,48,37,0
4,A002,R051,02-00-00,59 ST,2017-04-01 16:00:00,Saturday,20:00:00,408.0,2017-04-01,"NY CITY CENTRAL PARK, NY US",0.0,0.0,0,48,37,0


In [17]:
top_stations = top_ten_stations(df)

In [18]:
#top_stations.drop(['TMIN', 'TMAX'],inplace=True)

In [19]:
top_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265194 entries, 27855 to 1667011
Data columns (total 6 columns):
STATION          265194 non-null object
DATETIME         265194 non-null datetime64[ns]
TOTAL_TRAFFIC    265194 non-null float64
DOW              265194 non-null object
TIME             265194 non-null object
BAD_WEATHER      265194 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 11.1+ MB


In [20]:
def scheduler(day, time, df, weather_forecast=None):
    """
    day = day of deployment
    time = time interval of deployment
    df = turnstile dataset
    weather = {'Precip', 'Normal'}
    
    Returns: names of top ten stations sorted
    by mean total traffic for given day and time interval
    """
    
    day_dict = {'Monday':0, 
                'Tuesday':1, 
                'Wednesday':2, 
                'Thursday':3, 
                'Friday':4}
    time_dict = {'Early Morning': "04:00:00", 
                 'Late Morning': "08:00:00", 
                 'Afternoon': "12:00:00", 
                 'Evening':"16:00:00", 
                 'Night':"20:00:00"}
    weather = None
    if weather_forecast == 'Precip':
        weather = 1
    else:
        weather = 0
    day_and_time = df[(df['DOW'].str.lower() == day.lower()) &
                    (df['TIME'] == time_dict[time]) &
                    (df['BAD_WEATHER'] == weather)]
    
    g = day_and_time.groupby(['TIME', 'DATETIME','STATION'])\
    .sum()\
    .reset_index()\
    .groupby(['TIME', 'STATION'])\
    .mean()\
    .reset_index()\
    .sort_values(by='TOTAL_TRAFFIC', ascending=False)
    g['TOTAL_TRAFFIC'] = g['TOTAL_TRAFFIC'].apply(lambda x: round(x))
    g = g[['STATION', 'TOTAL_TRAFFIC']].reset_index(drop=True)[:10]
    
    # Return HTML in cell to display title and table
    g.columns = ['Station', 'Average Traffic']
    display(HTML("<h4>{0} from {1} to {2}</h4>".format(time, time_dict[time], str(int(time_dict[time][:2]) + 4) + time_dict[time][2:])))
    display(HTML(g.to_html()))

# WTWY Street Team Scheduler

## Instructions:  

* Declare day of the week

* List available times:
    * Early Morning: 4:00AM - 8:00AM
    * Late Morning: 8:00AM - 12:00PM
    * Afternoon: 12:00PM - 4:00PM
    * Evening: 4:00PM - 8:00PM
    * Night: 8:00PM - 12:00AM

* Declare weather forecast
    * "Precip" for rain or snow
    * Anything else for non-precipitation weather


In [21]:
day = "Friday"
time = "Late Morning"
weather = None

scheduler(day, time, top_stations, weather_forecast=weather)

,Station,Average Traffic
0,34 ST-PENN STA,46487
1,42 ST-GRD CNTRL,37713
2,FULTON ST,36912
3,42 ST-PORT AUTH,25360
4,14 ST-UNION SQ,25008
5,125 ST,19260
6,86 ST,18665
7,23 ST,17244
8,34 ST-HERALD SQ,15090
9,42 ST-TIMES SQ,12587
